<a href="https://colab.research.google.com/github/BrettAF/ERD/blob/main/Windowing_and_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries and variables

In [39]:
number_of_windows = 25200
window_size = 600
percent_testing=.1
percent_validation=.1
batch_size = 128
number_of_epochs=300

#This is the period before a seizure that will be examined.
Period_of_interest=60*60*1

In [2]:
import os
import pandas as pd
import numpy as np

from pathlib import Path
import tensorflow as tf
from tensorflow import keras
from keras import layers
from tensorflow.python import training

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)
import random as r
import scipy as sp
from scipy.stats import zscore
from scipy.fft import fft, ifft

from keras.layers.rnn import LSTM

In [40]:

time_between_signals = 3.906266
#This window size generates a number of samplings in a window of specific length
window_time= (window_size*time_between_signals)/1000

validation_windows=int(number_of_windows*(percent_validation))
testing_windows=int(number_of_windows*(percent_testing))
training_windows=number_of_windows-testing_windows-validation_windows

print("the Period of interest is",Period_of_interest/3600,"hour\n",
      "This setup will generate:\n",training_windows , " windows for training,\n",
      validation_windows,"windows for validation &\n",
      testing_windows,"Windows for testing.\n",
      "Each window has",window_time,"seconds, and",window_size,"frames\n",
      "This is about",number_of_windows*window_time/60/60,"hours of data")


the Period of interest is 1.0 hour
 This setup will generate:
 20160  windows for training,
 2520 windows for validation &
 2520 Windows for testing.
 Each window has 2.3437596 seconds, and 600 frames
 This is about 16.4063172 hours of data


# Reading & Shaping Data

In [5]:
dfs=[]
for x in os.listdir():
    if x.endswith("formated.csv"):
      print(x)
      dfs.append( pd.read_csv(str(x)))




chb01_03 formated.csv
chb01_01 formated.csv


In [6]:
#for recording the lables of the dataset as a one-hot vector
def LabelVector3(label):
  zero_vector=[1,0,0]
  one_vector =[0,1,0]
  two_vector =[0,0,1]

  if label==0:
      return zero_vector
  elif label ==1:
      return one_vector
  else: return two_vector

def LabelVector2(label):
  zero_vector=[1,0]
  one_vector =[0,1]
  if label==0:
      return zero_vector
  else :
      return one_vector

#This ensures that no two training windows are the same.
#It generates a vector of shuffled unique start locations
def vectorMakerV2():
  arr=[]
  for df in range(0,len(dfs)):
    for row in range(0,len(dfs[df])-window_size-1,window_size):
        arr.append([df,row])
  np.random.shuffle(arr)
  arr=np.array(arr)

  window_df_starts = list(arr[:,0])
  window_starts= list(arr[:,1])
  return window_starts,window_df_starts

In [7]:

def frameArrayMakerV3(i, window_length,  window_starts, window_df_starts,normalize,fft_flag, include_seizures):
  frames_array=[]
  label_array=[]
  label_OHV=[]

  while i<window_length:
    fft_array=[]
    start=window_starts[i]
    df=window_df_starts[i]
    i=i+1
    #Still ordered data from a dataframe of a specific size
    new_df=(dfs[df]).loc[start:start+window_size-1]

  #Stores the labels in an array
    #If the number is negative, it is a seizure and is marked with a 2
    if (new_df.at[start+window_size-1,"Label"] <0):
      new_label=2
    #if the label is 0, it is after the last siezure, i don;t really know how to treat it
    elif (new_df.at[start+window_size-1,"Label"] ==0):
      new_label = 0
    #if the label is greater than the poi, then it is far from the seizure and is marked with a 0
    elif(new_df.at[start+window_size-1,"Label"] >Period_of_interest):
      new_label=0
    #If none of the others were true, it must be in the period of interest, and it gets marked with a 1
    else:
      new_label=1


    if ((include_seizures == True) or (new_label != 2) ):
      new_df=new_df.drop(columns=["EEG Time", "Label"])
      # normalizes the columns
      if normalize == True:
          new_df = zscore(new_df)
      label_array.append(new_label)

      if (include_seizures == True):
          label_OHV.append( LabelVector3(new_label))
      else:
          label_OHV.append( LabelVector2(new_label))

        #if fft_flag == True:
          #for j in range(0,16):
            #one_channel=new_df[:,j]
            #anfft=fft(one_channel)
            #fft_array.append(anfft.real[20:window_size//2])
            #fft_array.append(anfft.imag[20:window_size//2])
          #new_df=fft_array
      frames_array.append( new_df)


  return frames_array, label_array, label_OHV,i

In [8]:
#This function guarantees that the number of each training label should stay aproximately even
def rebalanceRatio(training_ds_p, training_lables_p,training_labels_OHV_p):

  training_ds=training_ds_p[0:training_windows]
  training_lables=training_lables_p[0:training_windows]
  training_labels_OHV=training_labels_OHV_p[0:training_windows]

  ratio=training_lables.count(1)/training_windows
  print("ratio before adjusting:",ratio)
  #a moves through the tail
  #b moves through the training window

  a=training_windows
  b=0
  while ratio >.505 and(a<len(training_ds_p)-1)and(b<training_windows-1):
    while training_lables_p[a] !=0:
      a+=1
    while training_lables[b] !=1:
      b+=1
    training_lables[b]=training_lables_p[a]
    training_ds[b]=training_ds_p[a]
    training_labels_OHV[b]=training_labels_OHV_p[a]
    a+=1
    b+=1
    ratio=training_lables.count(1)/training_windows

  while ratio <.495 and(a<len(training_ds_p)-1)and(b<training_windows-1):
    while training_lables_p[a] !=1:
      a+=1
    while training_lables[b] !=0:
      b+=1
    training_lables[b]=training_lables_p[a]
    training_ds[b]=training_ds_p[a]
    training_labels_OHV[b]=training_labels_OHV_p[a]
    a+=1
    b+=1
    ratio=training_lables.count(1)/training_windows

  return training_ds,training_lables,training_labels_OHV


In [9]:
#This generates the windows for training, testing, and validation.
#If hot vectors it true it will return your labels in the form of a a one-hot-vector
#If print ratio is true, it will print the number of each type of label and their ratio
#if normalize is true, it will perform z-score normalization
def get_dataset_partitions_tf(dfs, hot_vector=True, print_ratio = True, normalize=False, fft=False, include_seizures = True):

    window_starts,window_df_starts= vectorMakerV2()

    #For testing data
    testing_ds, testing_lables,testing_labels_OHV,i =frameArrayMakerV3(0,testing_windows+1,  window_starts,window_df_starts,normalize,fft,include_seizures)

    #For validation data
    validation_ds, validation_lables,validatiion_labels_OHV,i =frameArrayMakerV3(i,validation_windows+testing_windows+1,  window_starts,window_df_starts,normalize,fft,include_seizures)

     #for training data
    training_ds_p, training_lables_p,training_labels_OHV_p,i =frameArrayMakerV3(i,len(window_starts), window_starts,window_df_starts,normalize,fft,include_seizures)
    #print(i,window_starts[30],window_df_starts[30])
    print('--Training Data--')

    training_ds, training_lables,training_labels_OHV = rebalanceRatio(training_ds_p, training_lables_p,training_labels_OHV_p)
    #0: not poi or seizure
    #1: period of interest
    #2: seizure



    if print_ratio == True:
      print("# of POI     :",training_lables.count(1),",",100*training_lables.count(1)/training_windows,'%')
      print("# of not POI :",training_lables.count(0),",",100*training_lables.count(0)/training_windows,'%')
      print("# of seizures:",training_lables.count(2),",",100*training_lables.count(2)/training_windows,'%','\n')
    if hot_vector == True:
      return training_ds, testing_ds, validation_ds, training_labels_OHV, testing_labels_OHV,validatiion_labels_OHV
    return training_ds, testing_ds, validation_ds, training_lables, testing_lables, validation_lables

In [10]:
#This generates the windows for training, testing, and validation.
#If hot vectors it true it will return your labels in the form of a a one-hot-vector
train_ds, test_ds, valid_ds, train_lables, test_lables,  valid_lables = get_dataset_partitions_tf(dfs,print_ratio=True, normalize=True, include_seizures=False, hot_vector=True)


--Training Data--
ratio before adjusting: 0.49166666666666664
# of POI     : 713 , 49.513888888888886 %
# of not POI : 727 , 50.486111111111114 %
# of seizures: 0 , 0.0 % 



In [11]:

training_data = np.array(train_ds)
testing_data = np.array(test_ds)
validation_data= np.array(valid_ds)


training_lables= np.array(train_lables)
testing_lables= np.array(test_lables)
validation_lables=np.array(valid_lables)


print("training  ",training_data.shape,",",training_lables.shape)
print("testing   ",testing_data.shape,",",testing_lables.shape)
print("validating",validation_data.shape,",",validation_lables.shape)
print('\n')


training   (1440, 600, 16) , (1440, 2)
testing    (180, 600, 16) , (180, 2)
validating (178, 600, 16) , (178, 2)




# Models & Training

In [35]:
learning_rate=tf.keras.optimizers.schedules.CosineDecayRestarts(initial_learning_rate=.01,
                                                                first_decay_steps=20, t_mul=2.0, m_mul=1.0, alpha=0.0, name=None)
opt = keras.optimizers.legacy.Adam(learning_rate=learning_rate)

In [ ]:
# normalized .5466,
input_shape = ( window_size, 16, 1)
training_data = training_data.reshape(-1,window_size, 16, 1)

print("training  ",training_data.shape,",",training_lables.shape)
m2Dlstm = keras.Sequential([
  layers.Conv2D(filters=32, kernel_size=(5,5) ,input_shape=input_shape),
  layers.Conv2D(filters=16, kernel_size=(3,3)),
  layers.MaxPool2D( pool_size=(11, 5)),
  layers.Reshape((54,32)),
  layers.LSTM(90, activation='relu' ),
  layers.Dense(75),
  layers.Dense(40),
  layers.Dense(training_lables.shape[1])
])
print('\n',m2Dlstm.summary())
m2Dlstm.compile(
              optimizer=opt,
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

m2Dlstm.fit(training_data, training_lables, batch_size=batch_size, epochs=number_of_epochs, validation_data=(validation_data, validation_lables))

# Evaluating


In [17]:
print('\n--Accuracy on testing data--')
test_loss, test_acc = m2Dlstm.evaluate(testing_data,testing_lables, verbose=2)
print( test_acc)



--Accuracy on testing data--
6/6 - 0s - loss: 0.6828 - accuracy: 0.5556 - 387ms/epoch - 64ms/step
0.5555555820465088


In [ ]:
m2Dlstm.save('.')